# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure, BackendWrapper
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `BackendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `BackendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = BackendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [4]:
print("| Does we have GPU backend to access:", backend.is_aer_gpu)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [13]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [8]:
from qurry.recipe.library import TrivialParamagnet, GHZ

In [12]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01.wave())
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02.wave())

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

In [14]:
experiment_execution_02.add(sample01.wave(), 'TrivialParamagnet')
experiment_execution_02.add(sample02.wave(), 'GHZ')
experiment_execution_02.add(sample02.wave())
experiment_execution_02.waves

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>

If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

In [15]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'a371d2f0-f25c-4022-992d-0fa2809fd8fa'

In [16]:
experiment_execution_02.exps

<ExperimentContainer={
    a371d2f0-f25c-4022-992d-0fa2809fd8fa: ...
} with 1 experiments load, a customized dictionary>

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

And we want to know the Renyi Entropy of half system. Then

In [17]:
experiment_execution_02.exps[exp1].analyze(
    (0, 4)
)

|  - 00:00 < ?

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.085250644683838, entropy=-0.11802827961166419, and others), 0 unused arguments>

- You will found a report in `reports`

In [18]:
experiment_execution_02.exps[exp1].reports

<AnalysisContainer={
    0: ...
} with 1 analysis load, a customized dictionary>

In [19]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [20]:
main

{'purity': 1.085250644683838,
 'entropy': -0.11802827961166419,
 'puritySD': 0.9219032140143772,
 'entropySD': 1.2255465606524565,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.460403881072998,
 'entropyAllSys': -0.5463674079016825,
 'puritySDAllSys': 1.907963766941096,
 'entropySDAllSys': 1.8848278208760503,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.20921850237621262,
 'mitigatedPurity': 0.7619556690267512,
 'mitigatedEntropy': 0.3922210314676187,
 'num_qubits': 8,
 'measure': 'measure range: (0, 8)',
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.00101097,
 'takingTimeAllSys': 0.003631155,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2023-12-08 21:15:04',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [21]:
side_prodct

{'purityCells': {94: 0.3366661071777344,
  10: 1.0396652221679688,
  98: 2.3355541229248047,
  38: 0.4231433868408203,
  58: 3.668916702270508,
  55: 1.451080322265625,
  23: 0.2035999298095703,
  4: 0.4166603088378906,
  29: 0.9898319244384766,
  50: 0.18678855895996094,
  45: 0.8540477752685547,
  7: 0.30043983459472656,
  52: 0.39267921447753906,
  67: 1.1649150848388672,
  74: 1.5161571502685547,
  39: 1.4526023864746094,
  56: 0.7681827545166016,
  17: 0.4980964660644531,
  59: 1.4511547088623047,
  46: 1.6836128234863281,
  33: 0.26143836975097656,
  73: 0.09354782104492188,
  32: 1.4813385009765625,
  44: 0.1337566375732422,
  8: 0.3845539093017578,
  22: 2.2182464599609375,
  53: 0.33995628356933594,
  65: 0.2037944793701172,
  84: 0.27897071838378906,
  90: 0.4830131530761719,
  24: 0.6211776733398438,
  28: 0.48456382751464844,
  87: 1.4054927825927734,
  78: 2.2416954040527344,
  89: 0.44782257080078125,
  92: 3.7884044647216797,
  36: 2.385547637939453,
  62: 2.953620910644

In [22]:
from qurry import BackendAvailabilities

BackendAvailabilities

{'Python': True, 'Rust': True}